### Load Template

In [3]:
import os
f = open("reg_optimization_template.py", "r")
script = f.readlines()


In [4]:
script[703]

'    index_train, index_test = surface_train_index, surface_test_index\n'

### Iterate over all regularization values and features 

In [ ]:
reg = [1e-10, 1e-8, 1e-5, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1000, 10000]
s_ref_list = [2, 4, 8, 16, 32, 64, 128]
b_ref_list = [8, 16, 32, 64, 128 ,256, 512]
t_ref_list = [8, 16, 32, 64, 128, 256, 512]

full_name = {'s' : "surface",
            'b' : "bulk",
            "t" : "total"}

short_name = ['s', 'b', 't']

registry = []

targets = {'s' : ["ldos3", "ldos1", "aligned_dos3", "aligned_dos1"],
          'b' : ["ldos3", "ldos1"],
           't' : ["ldos3", "ldos1", "aligned_dos3", "aligned_dos1"]
          }

surface_indexes = {"normal": "surface_train_index, surface_test_index",
                   "holdout": "surface_holdout_train_index, surface_holdout_test_index"
                  }

bulk_indexes = {"normal": "bulk_train_index, bulk_test_index",
                "biased": "bulk_biased_train_index, bulk_biased_test_index"}

total_indexes = {"normal": "total_train_index, total_test_index",
                 "biased": "total_biased_train_index, total_biased_test_index", 
                 "holdout": "holdout_train_index, holdout_test_index"
                }

models = ["Adam", "LBFGS", "Analytical"]
model_line = {"Analytical": [711,712],
              "Adam" : [713,714],
             "LBFGS" : [715,716]}


for model in model_line:
    new_script = script.copy()
    m_line = script[model_line[model][0] - 1]
    m_line = m_line[2:]
    new_script[model_line[model][0] - 1] = m_line
    for s_name in short_name:
        feat_path = "../../data/{}_soap.pt".format(full_name[s_name])
        for t in targets[s_name]:
            target_path = "../../data/{}_{}.pt".format(full_name[s_name], t)
            if '3' in t:
                spline_path = "../../data/splines_{}_03.pt".format(full_name[s_name])
            elif '1' in t:
                spline_path = "../../data/splines_{}_01.pt".format(full_name[s_name])
            else:
                raise ValueError('Not sure what is the spline path')
                for indices in globals()['{}_indexes'.format(full_name[s_name])]:
                    index_line = '    index_train, index_test = {}\n'.format(indices)
                    for regularization in reg:
                        regularization_line = '    regularization_value = {}\n'.format(regularization)

        

### Generate File

In [77]:
def edit_file(dir_name, feature_path, target_path, model, indices, reg, kernel, kernel_path):
    f = open("corr_train_template.py", "r")
    script = f.readlines()
    CHECK_FOLDER = os.path.isdir(dir_name)
    if not CHECK_FOLDER:
        os.makedirs(dir_name) 
    script[440] = '    y = torch.load("{}")\n'.format(target_path)
    script[441] = '    feats = torch.load("{}")\n'.format(feature_path)
    script[442] = '    index_train, index_test = {}\n'.format(indices)
    script[443] = '    regularization_value = {}\n'.format(reg)
    if kernel:
        script[444] = '    kMM = torch.load("{}")\n'.format(kernel_path)
        if model == "Adam":
            script[447] =  adam_kernel_train_line
        else:
            script[447] = lbfgs_kernel_train_line
    else:
        if model == "Adam":
            script[447] =  adam_soap_train_line
        else:
            script[447] = lbfgs_soap_train_line
            
    w = open(dir_name + "/script.py", "w")
    w.writelines(script)
    w.close()

In [82]:
i = 0
registry = []

features = surface_features
targets = surface_targets
indexes = surface_indexes
kMMs = surface_kMM
for feature_name in features:
    if 'kernel' in feature_name:
        kernel = True
        kernel_path = kMMs[feature_name]
    else:
        kernel = False
        kernel_path = None
    for target_name in targets:
        for indice in indexes:
            for model in models:
                try:
                    regu = regularizations[feature_name][target_name][indice]
                    dir_name = "./scripts/{}/".format(i)
                    edit_file(dir_name, features[feature_name], targets[target_name], model, indexes[indice], regu, kernel, kernel_path)
                    line = "For Model: {}, Feature: {}, Target: {}, index: {}, the directory is {}".format(model, feature_name, target_name, indice, i)
                    registry.append(line + "\n")
                    print (line)
                    print (regu)
                    print ('')
                    i+=1
                except:
                    continue
                    
features = bulk_features
targets = bulk_targets
indexes = bulk_indexes
kMMs = bulk_kMM
for feature_name in features:
    if 'kernel' in feature_name:
        kernel = True
        kernel_path = kMMs[feature_name]
    else:
        kernel = False
        kernel_path = None
    for target_name in targets:
        for indice in indexes:
            for model in models:
                try:
                    regu = regularizations[feature_name][target_name][indice]
                    dir_name = "./scripts/{}/".format(i)
                    edit_file(dir_name, features[feature_name], targets[target_name], model, indexes[indice], regu, kernel, kernel_path)
                    line = "For Model: {}, Feature: {}, Target: {}, index: {}, the directory is {}".format(model, feature_name, target_name, indice, i)
                    registry.append(line+ "\n")
                    print (line)
                    print (regu)
                    print ('')
                    i+=1
                except:
                    continue
                    
features = total_features
targets = total_targets
indexes = total_indexes
kMMs = total_kMM
for feature_name in features:
    if 'kernel' in feature_name:
        kernel = True
        kernel_path = kMMs[feature_name]
    else:
        kernel = False
        kernel_path = None
    for target_name in targets:
        for indice in indexes:
            for model in models:
                try:
                    regu = regularizations[feature_name][target_name][indice]
                    dir_name = "./scripts/{}/".format(i)
                    edit_file(dir_name, features[feature_name], targets[target_name], model, indexes[indice], regu, kernel, kernel_path)
                    line = "For Model: {}, Feature: {}, Target: {}, index: {}, the directory is {}".format(model, feature_name, target_name, indice, i)
                    registry.append(line+ "\n")
                    print (line)
                    print (regu)
                    print ('')
                    i+=1
                except:
                    continue


For Model: Adam, Feature: surface_soap, Target: surface_dos3, index: normal, the directory is 0
0.01

For Model: LBFGS, Feature: surface_soap, Target: surface_dos3, index: normal, the directory is 1
0.01

For Model: Adam, Feature: surface_soap, Target: surface_dos3, index: holdout, the directory is 2
0.1

For Model: LBFGS, Feature: surface_soap, Target: surface_dos3, index: holdout, the directory is 3
0.1

For Model: Adam, Feature: surface_soap, Target: surface_dos1, index: normal, the directory is 4
0.01

For Model: LBFGS, Feature: surface_soap, Target: surface_dos1, index: normal, the directory is 5
0.01

For Model: Adam, Feature: surface_soap, Target: surface_dos1, index: holdout, the directory is 6
0.1

For Model: LBFGS, Feature: surface_soap, Target: surface_dos1, index: holdout, the directory is 7
0.1

For Model: Adam, Feature: surface_soap, Target: surface_aligned_dos3, index: normal, the directory is 8
0.01

For Model: LBFGS, Feature: surface_soap, Target: surface_aligned_dos3,

For Model: LBFGS, Feature: total_kernel_150, Target: total_aligned_dos1, index: normal, the directory is 91
0.1

For Model: Adam, Feature: total_kernel_150, Target: total_aligned_dos1, index: biased, the directory is 92
0.1

For Model: LBFGS, Feature: total_kernel_150, Target: total_aligned_dos1, index: biased, the directory is 93
0.1

For Model: Adam, Feature: total_kernel_150, Target: total_aligned_dos1, index: holdout, the directory is 94
0.01

For Model: LBFGS, Feature: total_kernel_150, Target: total_aligned_dos1, index: holdout, the directory is 95
0.01



In [90]:
w = open("./bfscripts.registry", "w")
w.writelines(registry)
w.close()
    

In [89]:
registry = [i + "\n" for i in registry]

In [ ]:
dir_name = 0


In [87]:
dir_name

'./scripts/95/'